# GRU4REC모델로 영화 추천하기

## 1. 데이터 준비

In [1]:
import datetime as dt
from pathlib import Path
import os

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose-data/ml-1m') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


## 2. EDA

In [3]:
data = data[data['Rating'] >=3] ## Rating이 1 혹은 2는 비추천에 가깝기 때문에 제거한다.
data.head()

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103


In [4]:
data['UserId'].nunique(), data['ItemId'].nunique()

(6039, 3628)

In [5]:
user_length = data.groupby('UserId').size()
user_length

UserId
1        53
2       116
3        46
4        19
5       143
       ... 
6036    708
6037    189
6038     18
6039    119
6040    276
Length: 6039, dtype: int64

In [6]:
user_length.median(), user_length.mean()

(81.0, 138.51266766020865)

In [7]:
user_length.min(), user_length.max()

(1, 1968)

In [8]:
user_length.quantile(0.999)

1118.8860000000013

In [9]:
length_count = user_length.groupby(user_length).size()
length_percent_cumsum = length_count.cumsum() / length_count.sum()
length_percent_cumsum_999 = length_percent_cumsum[length_percent_cumsum < 0.999]

In [10]:
oldest, latest = data['Time'].min(), data['Time'].max()
print(oldest) 
print(latest)

956703932
1046454590


## 3. 데이터 전처리

In [11]:
def cleanse_recursive(data: pd.DataFrame, shortest, least_click) -> pd.DataFrame:
    while True:
        before_len = len(data)
        data = cleanse_short_session(data, shortest)
        data = cleanse_unpopular_item(data, least_click)
        after_len = len(data)
        if before_len == after_len:
            break
    return data


def cleanse_short_session(data: pd.DataFrame, shortest):
    session_len = data.groupby('UserId').size()
    session_use = session_len[session_len >= shortest].index
    data = data[data['UserId'].isin(session_use)]
    return data


def cleanse_unpopular_item(data: pd.DataFrame, least_click):
    item_popular = data.groupby('ItemId').size()
    item_use = item_popular[item_popular >= least_click].index
    data = data[data['ItemId'].isin(item_use)]
    return data

In [12]:
len(data)
data.head()
print(data.shape)

(836478, 4)


In [13]:
train = data[:int(len(data)*0.8)]
val = data[int(len(data)*0.8) : int(len(data)*0.9)]
test = data[int(len(data) * 0.9) :]

In [14]:
def stats_info(data: pd.DataFrame, status: str):
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Users: {data["UserId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {data["Time"].min()}\n'
          f'\t Last Time : {data["Time"].max()}\n')

In [15]:
stats_info(train, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 669182
	 Users: 4805
	 Items: 3600
	 First Time : 962935793
	 Last Time : 1046393499

* valid Set Stats Info
	 Events: 83648
	 Users: 640
	 Items: 3121
	 First Time : 959972581
	 Last Time : 1046454590

* test Set Stats Info
	 Events: 83648
	 Users: 596
	 Items: 3056
	 First Time : 956703932
	 Last Time : 1046437932



In [16]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(train['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

train = indexing(train, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

In [17]:
save_path = data_path / 'processed'
save_path.mkdir(parents=True, exist_ok=True)

train.to_pickle(save_path / 'train.pkl')
val.to_pickle(save_path / 'valid.pkl')
test.to_pickle(save_path / 'test.pkl')

## 4. 데이터 파이프라인 설계하기

In [18]:
class UserDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['UserId'].nunique())  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['UserId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('UserId').size().cumsum()
        return offsets

In [19]:
tr_dataset = UserDataset(train)
tr_dataset.df.head(10)

,UserId,ItemId,Rating,Time,item_idx
31,1,3186,4,978300019,0
22,1,1270,5,978300055,1
27,1,1721,4,978300055,2
37,1,1022,5,978300055,3
24,1,2340,3,978300103,4
36,1,1836,5,978300172,5
3,1,3408,4,978300275,6
7,1,2804,5,978300719,7
47,1,1207,4,978300719,8
0,1,1193,5,978300760,9


In [20]:
tr_dataset.click_offsets

array([     0,     53,    169, ..., 669083, 669112, 669182], dtype=int32)

In [21]:
tr_dataset.session_idx

array([   0,    1,    2, ..., 4802, 4803, 4804])

In [22]:
class UserDataLoader:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, dataset: UserDataset, batch_size=4):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드에서 확인해주세요.
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):  
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.
        
        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줍니다.
        mask = np.arange(self.batch_size)[(end - start) == 1]  
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록해줍니다.

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것입니다.
            new_session = last_session + i  
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것입니다.
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록합니다.
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += len(mask)  # 마지막 세션의 위치를 기록해둡니다.
        return start, end, mask, last_session, finished

In [23]:
tr_data_loader = UserDataLoader(tr_dataset, batch_size=4)
tr_dataset.df.head(15)

,UserId,ItemId,Rating,Time,item_idx
31,1,3186,4,978300019,0
22,1,1270,5,978300055,1
27,1,1721,4,978300055,2
37,1,1022,5,978300055,3
24,1,2340,3,978300103,4
36,1,1836,5,978300172,5
3,1,3408,4,978300275,6
7,1,2804,5,978300719,7
47,1,1207,4,978300719,8
0,1,1193,5,978300760,9


In [24]:
iter_ex = iter(tr_data_loader)

In [25]:
inputs, labels, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [ 0 53 64 54]
Label Item Idx are :       [ 1 54 62 24]
Previous Masked Input Idx are []


## 5. 데이터 평가지표 설계하기

In [26]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

## 6. 모델 설계하기

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [28]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=[predictions])
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

In [29]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = train['ItemId'].nunique()
        self.num_sessions = train['UserId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

args = Args(train, val, test, batch_size=64, hsz=50, drop_rate=0.1, lr=0.001, epochs=5, k=20)

In [30]:
model = create_model(args)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(64, 1, 3600)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 50), (64, 50)]      547800    
_________________________________________________________________
dropout (Dropout)            (64, 50)                  0         
_________________________________________________________________
dense (Dense)                (64, 3600)                183600    
Total params: 731,400
Trainable params: 731,400
Non-trainable params: 0
_________________________________________________________________


## 7. 데이터 학습시키기

In [31]:
# train 셋으로 학습하면서 valid 셋으로 검증합니다.
def train_model(model, args):
    train_dataset = UserDataset(args.tr)
    train_loader = UserDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.tr) - args.tr['UserId'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)
        for feat, target, mask in tr_loader:
            reset_hidden_states(model, mask)  # 종료된 session은 hidden_state를 초기화합니다. 아래 메서드에서 확인해주세요.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy = result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)  # valid set에 대해 검증합니다.

        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}\n")


def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  # model에서 gru layer를 가져옵니다.
    hidden_states = gru_layer.states[0].numpy()  # gru_layer의 parameter를 가져옵니다.
    for elt in mask:  # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  # parameter를 초기화 합니다.
    gru_layer.reset_states(states=hidden_states)


def get_metrics(data, model, args, k: int):  # valid셋과 test셋을 평가하는 코드입니다. 
                                             # train과 거의 같지만 mrr, recall을 구하는 라인이 있습니다.
    dataset = UserDataset(data)
    loader = UserDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = len(data) - data['UserId'].nunique()
    for inputs, label, mask in tqdm(loader, total=total_step // args.batch_size, desc='Evaluation', mininterval=1):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1)

        pred = model.predict(input_ohe, batch_size=args.batch_size)
        pred_arg = tf.argsort(pred, direction='DESCENDING')  # softmax 값이 큰 순서대로 sorting 합니다.

        length = len(inputs)
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])

    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

In [32]:
train_model(model, args)

Train:   0%|          | 0/10380 [00:00<?, ?it/s, accuracy=0, train_loss=5.84]     

	 - Recall@20 epoch 1: 0.236422
	 - MRR@20    epoch 1: 0.058557



Train:   0%|          | 0/10380 [00:00<?, ?it/s, accuracy=0.0156, train_loss=5.4] 

	 - Recall@20 epoch 2: 0.281277
	 - MRR@20    epoch 2: 0.075525



Train:   0%|          | 0/10380 [00:00<?, ?it/s, accuracy=0.0312, train_loss=5.33]

	 - Recall@20 epoch 3: 0.297791
	 - MRR@20    epoch 3: 0.082088



Train:   0%|          | 0/10380 [00:00<?, ?it/s, accuracy=0.0312, train_loss=5.46]

	 - Recall@20 epoch 4: 0.303704
	 - MRR@20    epoch 4: 0.085548



Evaluation:  89%|████████▉ | 1160/1297 [02:43<00:19,  7.08it/s]

	 - Recall@20 epoch 5: 0.305496
	 - MRR@20    epoch 5: 0.087492



## 8. 모델 평가하기

In [33]:
def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 20)
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")

test_model(model, args, test)

Evaluation:  89%|████████▉ | 1152/1297 [02:47<00:21,  6.89it/s]

	 - Recall@20: 0.301446
	 - MRR@20: 0.085223



노드에 사용했던 yoochoose데이터와 차이점은 데이터의 크기와 rating이 있다는 점이었다.     
yoochoose같은 경우 1달치의 데이터 크기가 100만 단위를 넘겼지만 movielens데이터에서의 데이터 전체 크기가 100만개가 조금 넘는 수준이었다.    
그리고 movielens에는 rating시스템이 있어 1점과 2점을 준 데이터는 사실상 비추천으로 분류하기 때문에 데이터를 지웠다.    
LongUser같은 경우에도 yoochoose데이터에서는 session 1개가 오래본 것이지만 movielens데이터는 ID기반이기 때문에 영화를 좋아하는 사람이라고 분류해 제거하지 않았다.    
accuracy가 엄청 높진 않지만 train_loss는 안정적으로 떨어졌으며 MRR, Recall수치는 지속적으로 개선되었다.

---

## 9. 하이퍼파라미터 수정하기 (dropout_rate를 0.3으로 올림)

In [34]:
rgs = Args(train, val, test, batch_size=32, hsz=50, drop_rate=0.3, lr=0.001, epochs=5, k=20)

In [35]:
model = create_model(args)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(64, 1, 3600)]           0         
_________________________________________________________________
GRU (GRU)                    [(64, 50), (64, 50)]      547800    
_________________________________________________________________
dropout_1 (Dropout)          (64, 50)                  0         
_________________________________________________________________
dense_1 (Dense)              (64, 3600)                183600    
Total params: 731,400
Trainable params: 731,400
Non-trainable params: 0
_________________________________________________________________


In [36]:
train_model(model, args)

Train:   0%|          | 0/10380 [00:00<?, ?it/s, accuracy=0.0156, train_loss=5.84]

	 - Recall@20 epoch 1: 0.238214
	 - MRR@20    epoch 1: 0.059214



Train:   0%|          | 0/10380 [00:00<?, ?it/s, accuracy=0.0312, train_loss=5.46]

	 - Recall@20 epoch 2: 0.281856
	 - MRR@20    epoch 2: 0.075130



Train:   0%|          | 0/10380 [00:00<?, ?it/s, accuracy=0.0469, train_loss=5.27]

	 - Recall@20 epoch 3: 0.296525
	 - MRR@20    epoch 3: 0.081715



Train:   0%|          | 0/10380 [00:00<?, ?it/s, accuracy=0.0625, train_loss=5.46]

	 - Recall@20 epoch 4: 0.302020
	 - MRR@20    epoch 4: 0.085140



Evaluation:  89%|████████▉ | 1160/1297 [02:47<00:19,  6.94it/s]

	 - Recall@20 epoch 5: 0.305159
	 - MRR@20    epoch 5: 0.087390



In [37]:
test_model(model, args, test)

Evaluation:  89%|████████▉ | 1152/1297 [02:53<00:21,  6.63it/s]

	 - Recall@20: 0.301961
	 - MRR@20: 0.084812



Dropout_rate를 0,1에서 0.3으로 올리니 train_loss가 기존 실험보다 확실하게 떨어지는 것을 볼 수 있다.    
그렇지만 MRR과 Recall수치는 조금 떨어지는 정도의 결과가 나왔다.

---

## 10. 하이퍼파라미터 수정하기 (k를 10으로 변경)

In [43]:
args = Args(train, val, test, batch_size=32, hsz=50, drop_rate=0.1, lr=0.001, epochs=5, k=10)

In [44]:
model = create_model(args)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(32, 1, 3600)]           0         
_________________________________________________________________
GRU (GRU)                    [(32, 50), (32, 50)]      547800    
_________________________________________________________________
dropout_3 (Dropout)          (32, 50)                  0         
_________________________________________________________________
dense_3 (Dense)              (32, 3600)                183600    
Total params: 731,400
Trainable params: 731,400
Non-trainable params: 0
_________________________________________________________________


In [45]:
train_model(model, args)

Train:   0%|          | 0/20761 [00:00<?, ?it/s, accuracy=0.0312, train_loss=5.26]

	 - Recall@10 epoch 1: 0.166137
	 - MRR@10    epoch 1: 0.060037



Train:   0%|          | 0/20761 [00:00<?, ?it/s, accuracy=0.0938, train_loss=5.66]

	 - Recall@10 epoch 2: 0.189394
	 - MRR@10    epoch 2: 0.072447



Train:   0%|          | 0/20761 [00:00<?, ?it/s, accuracy=0.0938, train_loss=5.68]

	 - Recall@10 epoch 3: 0.198340
	 - MRR@10    epoch 3: 0.077313



Train:   0%|          | 0/20761 [00:00<?, ?it/s, accuracy=0.0312, train_loss=5.64]

	 - Recall@10 epoch 4: 0.201899
	 - MRR@10    epoch 4: 0.079454



Evaluation:  95%|█████████▍| 2459/2594 [02:09<00:07, 18.92it/s]

	 - Recall@10 epoch 5: 0.202585
	 - MRR@10    epoch 5: 0.081215



In [46]:
test_model(model, args, test)

Evaluation:  94%|█████████▍| 2451/2595 [03:15<00:11, 12.51it/s]

	 - Recall@10: 0.299814
	 - MRR@10: 0.085725



k를 변경했는데 recall이 낮아졌음을 알 수 있다.    
top10개로 줄였기때문에 나온 결과 같다. 만일 k를 30으로 변경했으면 recall점수가 더 높을거라는 예측을 하게 된다.